In [1]:
import boto3
import json
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.34.138


# Configuration

In [2]:
boto3_session = boto3.session.Session()
bedrock = boto3.client(service_name='bedrock-runtime')

# Variables

In [3]:
model_id = "meta.llama3-70b-instruct-v1:0"

In [4]:
init_configuration = """You are an anonimization tool in identifying attributes in texts that can identify or quasi-identify a user."""

In [5]:
orden = """
Devuelve solo el texto original con la identificación y etiquetación del texto con la información personal del paciente añadiendolo entre claudators, por ejemplo: [**arquitecto**], [**8/9/21**], [**padre**]. No comentes nada más.
Las reglas que tienes que utilizar para la identifiación son en los siguientes 18 identificadores especificos en el ISP del HIPAA:
    "Nombres",
    "Datos geográficos",
    "Todos los elementos de las fechas",
    "Números telefónicos",
    "Números FAX",
    "Correos electrónicos",
    "Números de Seguridad Social",
    "Números de registros médicos",
    "Números de beneficiarios del plan de salud",
    "Números de cuenta",
    "Certificado / números de licencia",
    "Identificadores de vehículos y números de serie, incluidas placas",
    "Identificadores de dispositivo y números de serie",
    "URL web",
    "Direcciones de protocolo de Internet",
    "Identificadores biométricos (es decir, escaneo retiniano, huellas dactilares)",
    "Fotos de cara completa e imágenes comparables",
    "Cualquier número de identificación único, característica o código"
"""
#Las etiquetas posibles que puedes usar son [**NUMERO_TELEFONO**], [**FECHAS**], [**FAMILIARES_SUJETO_ASISTENCIA**], [**PROFESION**] y [**SÍNTOMAS**].  No puedes inventarte otras etiquetas.

In [6]:
with open('./data/processed/txt/CARMEN-I_CC_2.txt', 'r') as archivo:
    text = archivo.read()
    print(text)

Realizo llamada telefónica. Refiere buen descanso nocturno, sin incidencias. BEGLlamada:
Buen estado general, no tos, no disnea, no fiebre, no diarrea.
Alta el 20/05, cumpliéndose 3 semanas desde el incio de los síntomas.Visita a pacient
Tos: No
Dispnea: poc
SAt=2:97% basal
Ritme deposicional: normal
Temperatura: 36,4
EVA: 5/10 dolor pit
Deambula per l' habitacióPacient bon estat general.
Sense àlgies.
No diarrees.
Tarda sense incidències.Trucada de control:
Refereix bon descans nocturn.
No àlgies. No sensació distèrimica. No tos. No díspnea. No deposicions.EVOLUCIÓN Centro COVID
1. Probable neumonía bilateral Covid19
2.Síndrome ansioso
Situación social: vive con sus suegros que son mayores.
----
FECHA INICIO SÍNTOMAS: 27/04/2011
FECHA INGRESO URGENCIAS: 29/04/2011
FECHA INGRESO Hospital COVID: 3/05/2011
---
MEDICACIÓN COVID:
30/04: Azitromicina
---
OXÍGENO DURANTE LA ESTANCIA HOSPITALARIA: no
OXÍGENO AL ALTA/INGRESO HDOM: no
---
Analítica 7/05: PCR 0.40, LDH 177, d dimero 200, linfoci

In [7]:
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{init_configuration}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{orden}
{text}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [8]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an anonimization tool in identifying attributes in texts that can identify or quasi-identify a user.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Devuelve solo el texto original con la identificación y etiquetación del texto con la información personal del paciente añadiendolo entre claudators, por ejemplo: [**arquitecto**], [**8/9/21**], [**padre**]. No comentes nada más.
Las reglas que tienes que utilizar para la identifiación son en los siguientes 18 identificadores especificos en el ISP del HIPAA:
    "Nombres",
    "Datos geográficos",
    "Todos los elementos de las fechas",
    "Números telefónicos",
    "Números FAX",
    "Correos electrónicos",
    "Números de Seguridad Social",
    "Números de registros médicos",
    "Números de beneficiarios del plan de salud",
    "Números de cuenta",
    "Certificado / números de licencia",
    "Identificadores de vehículos y números de serie, incluidas placas",
  

In [9]:
max_gen_len = 1500
temperature = 0.1
top_p = 0.9

In [10]:
body = json.dumps({
    "prompt":prompt,
    "max_gen_len":max_gen_len,
    "temperature":temperature,
    "top_p":top_p
})

In [11]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [ ]:
response_body = json.loads(response.get('body').read())

In [ ]:
response_body['generation']

"\n\nRealizo llamada telefónica. Refiere buen descanso nocturno, sin incidencias. BEGLlamada:\nBuen estado general, no tos, no disnea, no fiebre, no diarrea.\nAlta el 20/05, cumpliéndose 3 semanas desde el incio de los síntomas.Visita a paciente [**con suegros**]\nTos: No\nDispnea: poc\nSAt=2:97% basal\nRitme deposicional: normal\nTemperatura: 36,4\nEVA: 5/10 dolor pit\nDeambula per l' habitacióPaciente bon estat general.\nSense àlgies.\nNo diarrees.\nTarda sense incidències.Trucada de control:\nRefereix bon descans nocturn.\nNo àlgies. No sensació distèrimica. No tos. No díspnea. No deposicions.EVOLUCIÓN Centro COVID\n1. Probable neumonía bilateral Covid19\n2.Síndrome ansioso\nSituación social: vive con sus [**suegros**] que son [**mayores**].\n----\nFECHA INICIO SÍNTOMAS: [**27/04/2011**]\nFECHA INGRESO URGENCIAS: [**29/04/2011**]\nFECHA INGRESO Hospital COVID: [**3/05/2011**]\n---\nMEDICACIÓN COVID:\n[**30/04**]: Azitromicina\n---\nOXÍGENO DURANTE LA ESTANCIA HOSPITALARIA: no\nOXÍGE

In [ ]:
print(f"Generated Text: {response_body['generation']}")
print(f"Prompt Token count:  {response_body['prompt_token_count']}")
print(f"Generation Token count:  {response_body['generation_token_count']}")
print(f"Stop reason:  {response_body['stop_reason']}")

Generated Text: 

Realizo llamada telefónica. Refiere buen descanso nocturno, sin incidencias. BEGLlamada:
Buen estado general, no tos, no disnea, no fiebre, no diarrea.
Alta el 20/05, cumpliéndose 3 semanas desde el incio de los síntomas.Visita a paciente [**con suegros**]
Tos: No
Dispnea: poc
SAt=2:97% basal
Ritme deposicional: normal
Temperatura: 36,4
EVA: 5/10 dolor pit
Deambula per l' habitacióPaciente bon estat general.
Sense àlgies.
No diarrees.
Tarda sense incidències.Trucada de control:
Refereix bon descans nocturn.
No àlgies. No sensació distèrimica. No tos. No díspnea. No deposicions.EVOLUCIÓN Centro COVID
1. Probable neumonía bilateral Covid19
2.Síndrome ansioso
Situación social: vive con sus [**suegros**] que son [**mayores**].
----
FECHA INICIO SÍNTOMAS: [**27/04/2011**]
FECHA INGRESO URGENCIAS: [**29/04/2011**]
FECHA INGRESO Hospital COVID: [**3/05/2011**]
---
MEDICACIÓN COVID:
[**30/04**]: Azitromicina
---
OXÍGENO DURANTE LA ESTANCIA HOSPITALARIA: no
OXÍGENO AL ALTA/ING

# Metricas

In [ ]:
text_generated = response_body['generation']

In [ ]:
print(text_generated)



Realizo llamada telefónica. Refiere buen descanso nocturno, sin incidencias. BEGLlamada:
Buen estado general, no tos, no disnea, no fiebre, no diarrea.
Alta el 20/05, cumpliéndose 3 semanas desde el incio de los síntomas.Visita a paciente [**con suegros**]
Tos: No
Dispnea: poc
SAt=2:97% basal
Ritme deposicional: normal
Temperatura: 36,4
EVA: 5/10 dolor pit
Deambula per l' habitacióPaciente bon estat general.
Sense àlgies.
No diarrees.
Tarda sense incidències.Trucada de control:
Refereix bon descans nocturn.
No àlgies. No sensació distèrimica. No tos. No díspnea. No deposicions.EVOLUCIÓN Centro COVID
1. Probable neumonía bilateral Covid19
2.Síndrome ansioso
Situación social: vive con sus [**suegros**] que son [**mayores**].
----
FECHA INICIO SÍNTOMAS: [**27/04/2011**]
FECHA INGRESO URGENCIAS: [**29/04/2011**]
FECHA INGRESO Hospital COVID: [**3/05/2011**]
---
MEDICACIÓN COVID:
[**30/04**]: Azitromicina
---
OXÍGENO DURANTE LA ESTANCIA HOSPITALARIA: no
OXÍGENO AL ALTA/INGRESO HDOM: no
--

In [ ]:
import re 

def calc_metrics(ground_truth, predictions):

    set_ground_truth = set(ground_truth)
    set_predictions = set(predictions)

    true_positives = set_ground_truth.intersection(set_predictions)
    false_positives = set_predictions.difference(set_ground_truth)
    false_negatives = set_ground_truth.difference(set_predictions)
    
    # Cálculo de métricas
    precision = len(true_positives) / (len(true_positives) + len(false_positives))
    recall = len(true_positives) / (len(true_positives) + len(false_negatives))
    f1 = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1




def evaluate(masked, generated):
    """ 
    Input: 
        - masked (str): Ground_truth text
        - generated(str): Text to be evaluated

    Output:
        - Precision, Recall and F1 (float)
    """
    ground_truth = re.findall(r'\[\*\*(.*?)\*\*\]', masked)
    predictions = re.findall(r'\[\*\*(.*?)\*\*\]', generated)
    print(ground_truth)
    print(predictions)
    predictions = predictions[0:3]+predictions[5:]

    return calc_metrics(ground_truth, predictions)



In [ ]:
with open('./data/processed/masked/CARMEN-I_CC_2.txt', 'r') as archivo:
    text_hoped = archivo.read()

In [ ]:
cal_met = evaluate(text_hoped, text_generated)
print(f"Precision: {cal_met[0]}")
print(f"Recall: {cal_met[1]}")
print(f"F1: {cal_met[2]}")

['20/05', 'Centro COVID', 'suegros', '27/04/2011', '29/04/2011', 'Hospital COVID', '3/05/2011', '30/04', '7/05', '7/05', '18/05', 'Centro COVID', 'suegros', '27/04/2011', '29/04/2011', '3/05/2011', '30/04', '7/05', '7/05', 'WYX/8408/5545']
['con suegros', 'suegros', 'mayores', '27/04/2011', '29/04/2011', '3/05/2011', '30/04', '7/05', '7/05', '18/05', 'suegros', 'mayores', '27/04/2011', '29/04/2011', '3/05/2011', '30/04', '7/05', '7/05']
Precision: 0.7777777777777778
Recall: 0.6363636363636364
F1: 0.7000000000000001


## Cosine similarity

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([text_hoped, text_generated])

cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

print(f"La similitud del coseno entre los textos es: {cosine_sim[0][0]}")

La similitud del coseno entre los textos es: 0.9718933124730674
